## 1. 小Q的歌单

1. 小Q有X首长度为A的不同歌曲和Y首长度为B的不同歌曲。现在小Q想组成一个长度为K的歌单，
2. 在每首歌曲只重复一次，不考虑顺序的情况下该歌单有多少种组合
3. X, Y <= 100, A != B 输出结果对 1000000007 取模
4. 测试用例:
        输入:
        5
        2 3 3 3
        输出:
        9
        
### 分析：

    设歌单中长度为A,B的歌曲数量分别为 i,j 则 i * A + j * B <= k and i <= X and j <= y
    对于每一组合法的歌单可以简记为(i,j)，此时直接根据题意算出该情况下的组合数 s(i,j) = c(x,i) * c(y,j)
    将所有合法的（i,j）找出，并且将total = total + s(i,j)

In [43]:
class solution():
    
    c = [[0] * 105 for _ in range(101)]
    init_c = False
    mod_num = 1000000007
    def __init__(self):
        '''
            initial the matrix which stored the combination numbers
        '''
        if not solution.init_c:
            solution.init_matrix()
            
    @classmethod
    def init_matrix(cls):
        cls.c[0][0] = 1
        for i in range(1, 101):
            cls.c[i][0] = 1
            for j in range(1, i+1):
                # c(n,k) = c(n-1, k-1) + c(n-1, k)
                # (a*10 + b) % m = ( a% m
                cls.c[i][j] = (cls.c[i-1][j-1] + cls.c[i-1][j]) % cls.mod_num 
        cls.init_c = True
    
    @classmethod
    def get_comb_num(cls, k, a, x, b, y):
        ans = 0
        for i in range(0, x+1):
            sa = i * a
            if sa <= k and (k -sa) % b == 0:
                
                j = (k-sa) // b
                if j <= y:
                    ans = (ans + (cls.c[x][i] * cls.c[y][j]) % cls.mod_num )% cls.mod_num
        return ans
    
s = solution()
for i in range(4):
    for j in range(4):
        print(s.c[i][j], end=' ')
    print("")

1 0 0 0 
1 1 0 0 
1 2 1 0 
1 3 3 1 


In [44]:
# try:
#     k = int(input("input k : "))
#     tmp = input("input a, x, b, y split by space : ")
#     a, x, b, y = list(map(int, tmp.split(' ')))
# except Exception as e:
#     print(e)
k = 5
a, x, b, y = 2, 3, 3, 3
ans = s.get_comb_num(k, a, x, b, y)
print(ans)

9


## 2.  安排机器

1. m个机器，每个机器最长工作时间zi, 机器等级wi
2. n个任务，每个任务耗时 xi, 难度 yi
3. 一个任务只能由一个机器完成， 且 yi <= wi and xi <= cur(zi)
4. 完成一个任务的收益 200 * xi + 3 * yi
5. 求一天内完成任务的最大数量以及最大收益

        输入
        1 2 
        100 3
        100 2 
        100 1
        输出
        1 20006
        
### 分析
因为只求一天的任务数量，时间收益200 > 难度收益3， 而且每个机器只完成一个任务，

我们找到空闲时间大于任务的机器，并从它们之中选择难度刚刚好大于等于任务难度去执行该任务

用一个cnt数组去记录符合时间要求的机器编号，由于机器编号是按照最大时间排序的，任务也是，所以能满足第i任务的机器同时适用于i+1， i+2 ..之后的任务，虽然这种做法有点浪费，所以我们才执行 for k in range(jobs[i][1], 100) 去找到能满足任务要求但是配置最低的机器去执行它。

In [76]:
def arrange_job(workers, jobs):
    '''
        workers, jobs are lists which contain the information of time and difficult
        e.g workers =[[100,3],[100,2]]
    '''
    n , m = len(workers), len(jobs)
    if n * m == 0:
        return 0

    # sort descending
    workers.sort(key = lambda x: [x[0],x[1]], reverse=True)
    jobs.sort(key = lambda x: [x[0],x[1]], reverse=True)

    #Minimum requirements
    cnt = [0] * 105
    ans, num =0, 0
    j = 0
    for i in range(m):
       
        while j < n and workers[j][0] >= jobs[j][0]:
            #cnt records the workers_idx in which working time is greater than job_time
            if workers[j][1] >= jobs[i][1]:
                cnt[workers[j][i]] += 1
            j += 1
    
        for k in range(jobs[i][1], 101):
            #find the first worker which can handle the job
            if cnt[k]:
                num += 1
                cnt[k] -= 1
                ans +=  200 * jobs[i][0] + 3 * jobs[i][1]
                break
    print("{} {}".format(num, ans))
    return 0

In [75]:
workers=[[100,3]]
jobs = [[100,1],[100,2]]
arrange_job(workers, jobs)

[100, 2]
1 20006


0

## 3. 画家小Q

1. 小Q的画长度为 N * M,(N,M <= 50) 用X,B,Y,G 分别表示颜色白，蓝，黄，绿
2. 小Q每一笔可以以斜率为1斜线画一段并用B填充，或者以斜率为-1斜线画一段用Y填充
3. B + Y -> G
4. 小Q最少用几笔画完给定的画
5. 实例：
        输入
        YXXB
        XYGX
        XBYY
        BXXY
        输出
        3
        
### 分析

由于斜线的只有两种（左斜蓝色，右斜黄色，相交绿色），所以只需枚举即可。

In [41]:
class solution():
    
    def __init__(self):
        self.matrix = [[0]* 60 for _ in range(60)]
        self.m = self.n = 0
        
    def dfs_eliminate_line(self, x, y, color, slope):
        '''
        eliminate the special slope-color line start from (x,y) base on deep first search 
        
        @param x, y : int : coordinate
        @param color : str
        @param slope : int (1,1) or (1,-1)
        '''
        open_list = [(x,y)]
        close_set = set()
        close_set.add((x,y))
        other_color = 'B' if color != 'B' else 'Y'
        while open_list:
            x,y = open_list.pop()
            if 0 <= x and x < self.n and 0 <= y and y < self.m:
                if self.matrix[x][y] == 'G':
                    self.matrix[x][y] = other_color
                elif self.matrix[x][y] == color:
                    self.matrix[x][y] = 'X'
                next_one = [((x+slope[0]*i), (y+slope[1]*i)) for i in [1,-1]]
                for one in next_one:
                    x,y = one
                    if 0 <= x and x < self.n and 0 <= y and y < self.m:
                        if one not in close_set:
                            open_list.append(one)
                            close_set.add(one)
        
    def painting(self):
        cnt = 0
        for i in range(self.n):
            for j in range(self.m):
                if self.matrix[i][j] == 'Y':
                    self.dfs_eliminate_line(i, j, 'Y', (1,1))
                    cnt += 1
                elif self.matrix[i][j] =='B':
                    self.dfs_eliminate_line(i, j, 'B', (1,-1))
                    cnt += 1
                elif self.matrix[i][j] =='G':
                    self.dfs_eliminate_line(i, j, 'Y', (1,1))
                    self.dfs_eliminate_line(i, j, 'B', (1,-1))
                    cnt += 2
        print(cnt)
        return cnt
        
    def test(self):
        test=\
            [['Y', 'X', 'X', 'B'],
             ['X', 'Y', 'G', 'X'],
             ['X', 'B', 'Y', 'Y'],
             ['B', 'X', 'X', 'Y']]
        self.n, self.m = len(test), len(test[0])
        self.matrix = test
        
        self.painting()

In [42]:
s=solution()

s.test()
s.matrix

3


[['X', 'X', 'X', 'X'],
 ['X', 'X', 'X', 'X'],
 ['X', 'X', 'X', 'X'],
 ['X', 'X', 'X', 'X']]

## 4. 贪吃的小Q

1. n 天，m个巧克力
2. 每天吃巧克力数量 e(t) 不少于 前一天e(t-1)的一半, 既 e(t) >= ( e(t-1) + 1 ) / 2
3. 第一天最多吃多少巧克力
4. 例子

    输入：3 7
    输出: 4

### 分析

设第三天吃a块，则第二天吃 2*a, 第一天吃 4a 4a + 2a + a <= 7 解得 a =1, 第一天吃 4a = 4

问题当 n=3, 7 < m < 14 时，有余数，所以只能搜索



In [43]:
def firstDay(n, m):
    l, r = 1, m
    ok = None
    while l != r:
        ok= True
        mid = (l + r + 1) >> 1
        remain, now = m, mid
        # Can you hold on for n days:
        for j in range(n):
            if remain < now:
                ok = False
                break
            remain -= now
            now = (now + 1) >> 1
        if ok:
            # can you can more chocolate
            l = mid
        else:
            # oh, you get too much
            r = mid - 1
    return l

firstDay(3,7)

4

## 5. 纸牌游戏

1. n张牌，甲乙轮流抽牌， 甲前
2. 两人尽可能抽最大的牌
3. 求两人拥有的牌点数之差

### 分析
直接排序，奇数归甲，偶数归乙，直接计算即可

In [64]:
def get_difference(points):
    points.sort(reverse=True)
    s = 0
    for i, v in enumerate(points):
        if i & 1 == 0:
            s += v
        else:
            s -= v
    return s

In [65]:
points = [2, 7, 4]
get_difference(points)

5